# Baseline
python: 3.8.*

notebook0
## PART 0
難度：★★★★☆

In [ ]:
!cd /content

/bin/bash: line 0: cd: /content: No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 1st 執行

# !git clone https://github.com/s109703023/NCKU-AICUP2023-baseline.git
# !cd /content/drive/MyDrive/NCKU-AICUP2023-baseline/
# !gdown --folder 1tdf_i6XNYT6jpOA6F_lqU4mRRh1xYPcl
# !mv baseline/* ./

In [ ]:
### 2nd 之後
!cd /content/drive/MyDrive/NCKU-AICUP2023-baseline

/bin/bash: line 0: cd: /content/drive/MyDrive/NCKU-AICUP2023-baseline: No such file or directory


In [ ]:
ls 

In [ ]:
ls data

In [ ]:
%pip install -r requirements.txt

notebook1
## PART 1. Document retrieval
難度：★★★★☆

In [ ]:
!pip install -U ckiptagger[tf,gdown]

from ckiptagger import WS, POS, NER, data_utils, construct_dictionary


In [ ]:
# 下載模型檔 (1st執行)
# data_utils.download_data_gdown("./")

In [ ]:

# 載入模型
ws = WS("./data")
pos = POS("./data")
ner = NER("./data")

/usr/local/lib/python3.10/dist-packages/ckiptagger/model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/usr/local/lib/python3.10/dist-packages/ckiptagger/model_ner.py:57: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)


### Document retrieval 流程
- Step 1: 使用 Constituency Parser 找出 claim 中的 Noun Phrases (NPs）
- Step 2: 從 Wikipedia API 中取出和 NP 相符合的頁面名稱
- Step 3: 保留出現在句子 index 最靠前的五篇文章作為相關文章

Prepare the environment and import all library we need

In [ ]:
# built-in libs
import json
import pickle
import re
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Set, Tuple, Union

# 3rd party libs
import hanlp
import opencc
import pandas as pd
import wikipedia
from hanlp.components.pipeline import Pipeline
from pandarallel import pandarallel

# our own libs
from utils import load_json

pandarallel.initialize(progress_bar=True, verbose=0, nb_workers=10)
wikipedia.set_lang("zh")

Preload the data.

In [ ]:
TRAIN_DATA = load_json("data/public_train.jsonl")
TEST_DATA = load_json("data/private+public_test_data.jsonl")
CONVERTER_T2S = opencc.OpenCC("t2s.json")
CONVERTER_S2T = opencc.OpenCC("s2t.json")

In [ ]:
print(len(TRAIN_DATA))
print(len(TEST_DATA))

11620
9038


Data class for type hinting

In [ ]:
@dataclass
class Claim:
    data: str

@dataclass
class AnnotationID:
    id: int

@dataclass
class EvidenceID:
    id: int

@dataclass
class PageTitle:
    title: str

@dataclass
class SentenceID:
    id: int

@dataclass
class Evidence:
    data: List[List[Tuple[AnnotationID, EvidenceID, PageTitle, SentenceID]]]

### Helper function

For the sake of consistency, we convert traditional to simplified Chinese first before converting it back to traditional Chinese.  This is due to some errors occuring when converting traditional to traditional Chinese.

In [ ]:
def do_st_corrections(text: str) -> str:
    simplified = CONVERTER_T2S.convert(text)

    return CONVERTER_S2T.convert(simplified)

We use constituency parsing to separate part of speeches or so called constituent to extract noun phrases.  In the later stages, we will use the noun phrases as the query to search for relevant documents.  

![con](https://www.baeldung.com/wp-content/uploads/sites/4/2020/06/constituency_parse_tree-1.png)

##### 現在正在調這裡

In [ ]:
def get_nps_hanlp(
    predictor: Pipeline,
    d: Dict[str, Union[int, Claim, Evidence]],
) -> List[str]:
    claim = d["claim"]
    tree = predictor(claim)["con"]
    nps = [
        do_st_corrections("".join(subtree.leaves()))
        for subtree in tree.subtrees(lambda t: t.label() == "NP")
    ]

    return nps

In [ ]:
# ## add: 增加filter，刪除list中重複的元素（刪掉短的元素）
# def filter_nps(nps: List[str])-> List[str]:
#     n = len(nps)
#     if n==0:
#         return []
#     result = [nps[0]]
#     for s in nps:
#         if s not in result[len(result)-1]:
#             result.append(s)
#     return result

In [ ]:
def filter_nps(nps: List[str], bound: int)-> List[str]:
    n = len(nps)
    if n==0:
        return []

    result = [nps[0]]
    for i in range(n):
        ## filter_nps要取出NER(ckip tagger) 再去查維基百科
        sentence_list = [nps[i]]
        word_sentence_list = ws(sentence_list)
        pos_sentence_list = pos(word_sentence_list)
        entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
        for item in entity_sentence_list[0]:
              if item[2] in ['CARDINAL', 'DATE']:
                  continue
              if item[3] not in result:
                  result.append(item[3])

        if len(nps[i])>=bound:
            if nps[i] not in result:
                result.append(nps[i])

        else:
            if any(nps[i] in s for s in result):
                continue
            result.append(re.sub(r'[^\w\s]', '', nps[i]))

    return result


In [ ]:
## testt
list1 = ['天衛三軌道在天王星內部的磁層', '天衛三軌道', '天衛', '軌道', '天王星內部', '磁層', '《仲夏夜之夢》作者緹坦妮雅', '《仲夏夜之夢》作者', '《仲夏夜之夢》', '仲夏夜', '夢', '作者', '緹坦妮雅']
list2 = ['1917年俄羅斯人亞歷山大·克倫斯基領導了民主革命推翻了沙皇的支配，卻在同年又被推翻。', '俄羅斯人', '亞歷山大·克倫斯基']
print(filter_nps(list2,5))

['1917年俄羅斯人亞歷山大·克倫斯基領導了民主革命推翻了沙皇的支配，卻在同年又被推翻。', '亞歷山大·克倫斯基', '俄羅斯人', '民主革命', '沙皇']


In [ ]:
sentence_list = [list1[0]]
word_sentence_list = ws(sentence_list)
pos_sentence_list = pos(word_sentence_list)
entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

print(word_sentence_list)
print(pos_sentence_list)
print(entity_sentence_list[0])
print(type(entity_sentence_list[0]))
for item in entity_sentence_list[0]:
    print(item[3])

[['天衛', '三', '軌道', '在', '天王星', '內部', '的', '磁層']]
[['Nb', 'Neu', 'Na', 'P', 'Nb', 'Ncd', 'DE', 'Na']]
{(0, 2, 'LOC', '天衛'), (6, 9, 'LOC', '天王星')}
<class 'set'>
天衛
天王星


#### 評估 Document Retrieval

\begin{align*}
\textrm{Precision}&=\frac{\textrm{relevant} \cap \textrm{retrieved}}{\textrm{retrieved}} \\
&=\frac{\textrm{抽取到的文章與正確文章的交集}}{\textrm{抽取到的文章}}
\end{align*}

\begin{align*}
\textrm{Recall}&=\frac{\textrm{relevant} \cap \textrm{retrieved}}{\textrm{relevant}} \\
&=\frac{\textrm{抽取到的文章與正確文章的交集}}{\textrm{正確文章}}
\end{align*}

<img src="https://i.imgur.com/1q3c6Zw.png" alt="example" width="60%">

In [ ]:
def calculate_precision(
    data: List[Dict[str, Union[int, Claim, Evidence]]],
    predictions: pd.Series,
) -> None:
    precision = 0
    count = 0

    for i, d in enumerate(data):
        if d["label"] == "NOT ENOUGH INFO":
            continue

        # Extract all ground truth of titles of the wikipedia pages
        # evidence[2] refers to the title of the wikipedia page
        gt_pages = set([
            evidence[2]
            for evidence_set in d["evidence"]
            for evidence in evidence_set
        ])

        predicted_pages = predictions.iloc[i]
        hits = predicted_pages.intersection(gt_pages)
        if len(predicted_pages) != 0:
            precision += len(hits) / len(predicted_pages)

        count += 1

    # Macro precision
    print(f"Precision: {precision / count}")


def calculate_recall(
    data: List[Dict[str, Union[int, Claim, Evidence]]],
    predictions: pd.Series,
) -> None:
    recall = 0
    count = 0

    for i, d in enumerate(data):
        if d["label"] == "NOT ENOUGH INFO":
            continue

        gt_pages = set([
            evidence[2]
            for evidence_set in d["evidence"]
            for evidence in evidence_set
        ])
        predicted_pages = predictions.iloc[i]
        hits = predicted_pages.intersection(gt_pages)
        recall += len(hits) / len(gt_pages)
        count += 1

    print(f"Recall: {recall / count}")

The default amount of documents retrieved is at most five documents.  This `num_pred_doc` can be adjusted based on your objective.  Save data in jsonl format.

In [ ]:
def save_doc(
    data: List[Dict[str, Union[int, Claim, Evidence]]],
    predictions: pd.Series,
    mode: str = "train",
    num_pred_doc: int = 5,
) -> None:
    with open(
        f"data/{mode}_doc{num_pred_doc}.jsonl",
        "w",
        encoding="utf8",
    ) as f:
        for i, d in enumerate(data):
            d["predicted_pages"] = list(predictions.iloc[i])
            f.write(json.dumps(d, ensure_ascii=False) + "\n")

### Main function for document retrieval

In [ ]:
def overlap(BOW, str):
  res = 0
  for c in str:
    res += BOW[c]
  return 1/(res+1) # +1 to avoid 0 case

In [ ]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,3))

def get_pred_pages(series_data: pd.Series) -> Set[Dict[int, str]]:
    results = []
    claim = series_data["claim"]
    nps = series_data["hanlp_results"]
    first_wiki_term = []
    segment_claim = [' '.join(ws(claim)[0])]

    for i, np in enumerate(nps):
        # Simplified Traditional Chinese Correction
        wiki_search_results = [
            do_st_corrections(w) for w in wikipedia.search(np)
        ]

        for term in wiki_search_results:
            matched = False
            if i == 0:
                first_wiki_term.append(term)

            if (((new_term := term) in claim) or
                ((new_term := term.replace("·", "")) in claim) or
                ((new_term := term.split(" ")[0]) in claim) or
                ((new_term := term.replace("-", "")) in claim)):
                matched = True

            elif "·" in term:
                splitted = term.split("·")
                for split in splitted:
                    if (new_term := split) in claim:
                        matched = True
                        break

            if matched:
                term = term.replace(" ", "_")
                term = term.replace("-", "")
                results.append(term)

    # 10 is a hyperparameter
    if len(results) > 10:
        feature_vector = tfidf_vectorizer.fit_transform(segment_claim + results)
        cosine_similarities = cosine_similarity(feature_vector[0], feature_vector[1:]).tolist()[0]
        sorted_pairs = sorted(zip(results, cosine_similarities), key=lambda x: x[1], reverse=True)
        sorted_values = [pair[0] for pair in sorted_pairs][:10]
    if len(results) < 1:
        results = first_wiki_term[:1]

    return set(results)

#### Groupby pages
- 如果有多個後綴類型的pages，只取第一個配對到的
![groupby](https://imgur.com/FRE4BVv.png)

### Step 1. Get noun phrases from hanlp consituency parsing tree

Setup [HanLP](https://github.com/hankcs/HanLP) predictor (1 min)

In [ ]:
predictor = (hanlp.pipeline().append(
    hanlp.load("FINE_ELECTRA_SMALL_ZH"),
    output_key="tok",
).append(
    hanlp.load("CTB9_CON_ELECTRA_SMALL"),
    output_key="con",
    input_key="tok",
))

We will skip this process which for creating parsing tree when demo on class

##### 刪


In [ ]:
hanlp_file = f"data/hanlp_con_results.pkl"
if Path(hanlp_file).exists():
    with open(hanlp_file, "rb") as f:
        hanlp_results = pickle.load(f)
else:
    #hanlp_results = [get_nps_hanlp(predictor, d) for d in TRAIN_DATA]
    hanlp_results = [filter_nps(get_nps_hanlp(predictor, d),5) for d in TRAIN_DATA]
    with open(hanlp_file, "wb") as f:
        pickle.dump(hanlp_results, f)

Get pages via wiki online api

In [ ]:
# https://stackoverflow.com/questions/18603270/progress-indicator-during-pandas-operations
from tqdm import tqdm

tqdm.pandas()

In [ ]:
doc_path = f"data/train_doc5.jsonl"
if Path(doc_path).exists():
    with open(doc_path, "r", encoding="utf8") as f:
        predicted_results = pd.Series([
            set(json.loads(line)["predicted_pages"])
            for line in f
        ])
else:
    train_df = pd.DataFrame(TRAIN_DATA)
    train_df.loc[:, "hanlp_results"] = hanlp_results
    predicted_results = train_df.progress_apply(get_pred_pages, axis=1)
    save_doc(TRAIN_DATA, predicted_results, mode="train")

In [ ]:
## testt
print(TRAIN_DATA[0:4])
for d in TRAIN_DATA:
  if "1917" in d["claim"]:
  
    print(d["claim"])
    # print(d["predicted_pages"])
    print()
  # print(d["predicted_pages"])

[{'id': 2663, 'label': 'refutes', 'claim': '天衛三軌道在天王星內部的磁層，以《 仲夏夜之夢 》作者緹坦妮雅命名。', 'evidence': [[[4209, 4331, '天衛三', 2]]]}, {'id': 2399, 'label': 'refutes', 'claim': '信天翁科的活動範圍位於北冰洋以及南太平洋，牠的翼展可達到3.7米，是世界上現存的翼展最大的鳥類。', 'evidence': [[[2719, 2928, '信天翁科', 2]]]}, {'id': 8075, 'label': 'NOT ENOUGH INFO', 'claim': 'F.I.R. 的 團員有主唱Faye飛 （ 詹雯婷 ） 、 吉他手Real阿沁 （ 黃漢青 ） 、 鍵盤手Ian （ 陳建寧 ） ，是亞洲樂壇不常見的一女二男三人組合樂團 。', 'evidence': [[7208, None, None, None]]}, {'id': 8931, 'label': 'NOT ENOUGH INFO', 'claim': '香港國際機場全年24小時運作，它從2001年起一直躋身世界最佳機場 ， 並8度獲評級爲全宇宙最佳機場 。', 'evidence': [[8162, None, None, None]]}]
1917年俄羅斯人亞歷山大·克倫斯基領導了民主革命推翻了沙皇的支配，卻在同年又被推翻。

1917年俄羅斯人亞歷山大·克倫斯基領導了民主革命推翻了沙皇的統治，卻在同年又被推翻。

1917年俄羅斯人亞歷山大·克倫斯基作爲領導推翻了沙皇的統治，卻在同年又被推翻。

1917年俄羅斯政治家 、 革命家亞歷山大·克倫斯基領導了民主革命推翻了沙皇的統治，卻在同年又被推翻。

1917年領導了民主革命推翻了沙皇的統治，卻在同年又被推翻的亞歷山大·克倫斯基是俄羅斯人。

臺南市立建興國民中學校內有一棟歷史建築，建築於1917年。



In [ ]:
## testt
# for d in TRAIN_DATA:
for d in TRAIN_DATA[:10]:
  print(get_nps_hanlp(predictor, d))
  print(filter_nps(get_nps_hanlp(predictor, d),5))
  print()

### Step 2. Calculate our results

##### 結果！！

<table>
  <thead>
    <tr>
      <th> No. </th>
      <th> Version </th>
      <th> Precision </th>
      <th> Recall </th>
      <th> 說明 </th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1.</td>
      <td>原始版本</td>
      <td>0.25093</td>
      <td>0.80733</td>
      <td></td>
    </tr>
    <tr>
      <td>2.</td>
      <td>filter_nps</td>
      <td>0.41740</td>
      <td>0.76800</td>
      <td>濾掉所有短的重複字串</td>
    </tr>
    <tr>
      <td>3.</td>
      <td>filter_nps</td>
      <td>0.37931</td>
      <td>0.80656</td>
      <td>保留長度五個字以上的字串</td>
    </tr>
    <tr>
      <td>4.</td>
      <td>filter_nps</td>
      <td>0.32455</td>
      <td>0.82584</td>
      <td>用ckip加入ner</td>
    </tr>
    <tr>
      <td>5.</td>
      <td>main_func</td>
      <td>0.31097</td>
      <td>0.84584</td>
      <td>改wiki搜尋的權重(李的main func.)</td>
    </tr>
  </tbody>
</table>

In [ ]:
calculate_precision(TRAIN_DATA, predicted_results)
calculate_recall(TRAIN_DATA, predicted_results)

Precision: 0.3089940476190494
Recall: 0.8447552083333336


### Step 3. Repeat the same process on test set
Create parsing tree

In [ ]:
hanlp_file = f"data/hanlp_con_test_results.pkl"
hanlp_test_file = f"data/hanlp_con_test_results.pkl"
if Path(hanlp_test_file).exists():
    with open(hanlp_file, "rb") as f:
        hanlp_results = pickle.load(f)
else:
    # hanlp_results = [get_nps_hanlp(predictor, d) for d in TEST_DATA]
    hanlp_results = [filter_nps(get_nps_hanlp(predictor, d),5) for d in TEST_DATA]
    with open(hanlp_file, "wb") as f:
        pickle.dump(hanlp_results, f)

Get pages via wiki online api

In [ ]:
from tqdm import tqdm

In [ ]:
test_doc_path = f"data/test_doc5_private.jsonl"
if Path(test_doc_path).exists():
    with open(test_doc_path, "r", encoding="utf8") as f:
        test_results = pd.Series(
            [set(json.loads(line)["predicted_pages"]) for line in f])
else:
    test_df = pd.DataFrame(TEST_DATA)
    test_df.loc[:, "hanlp_results"] = hanlp_results
    test_results = pd.Series([
      get_pred_pages(row)
      for _, row in tqdm(test_df.iterrows(), desc="Processing", total=len(test_df))
    ])
    save_doc(TEST_DATA, test_results, mode="test")

Processing: 100%|██████████| 9038/9038 [2:16:40<00:00,  1.10it/s]


notebook2
## PART 2. Sentence retrieval
難度：★★★☆☆ (最好 ／‵Д′)／~ ╧╧ )


### 選擇Training Data (依照Part1 生成的1,2,3)

參數集中輸入


In [1]:
Use_Data_From_Part_1 = "5"  #@param {type:"string"}

### 選擇Training Data (依照Part1 生成的1,2,3,4)


In [5]:
# local libs
from utils import (
    generate_evidence_to_wiki_pages_mapping,
    jsonl_dir_to_df,
    load_json,
    load_model,
    save_checkpoint,
    set_lr_scheduler,
)

In [6]:
### Trainning Data From Part1

### always in form : data/train_doc5_data{x}.jsonl

File_From_Part_1 = f"data/train_doc5_data{Use_Data_From_Part_1}.jsonl"  

print("Use Training Data ",File_From_Part_1)

DOC_DATA = load_json(File_From_Part_1)   


## 缺檔案時報錯



Use Training Data  data/train_doc5_data4.jsonl


### Sentence retrieval 前置作業

內含:

* 調 Train/Dev 比例

* 轉換Wiki的code (3min)




使用方式 : 

- 蓋起來直接執行就好

注意 : 

- data/public_train.jsonl
- data/public_test.jsonl
有沒有加入第二階段資料


In [7]:
# built-in libs
from pathlib import Path
from typing import Dict, List, Set, Tuple, Union

# third-party libs
import numpy as np
import pandas as pd
from pandarallel import pandarallel
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_scheduler,
)

from dataset import BERTDataset, Dataset



pandarallel.initialize(progress_bar=True, verbose=0, nb_workers=10)

Global variable

In [11]:
SEED = 42

TRAIN_DATA = load_json("data/public_train.jsonl")
TEST_DATA = load_json("data/private_test_data.jsonl")

 
LABEL2ID: Dict[str, int] = {
    "supports": 0,
    "refutes": 1,
    "NOT ENOUGH INFO": 2,
}
ID2LABEL: Dict[int, str] = {v: k for k, v in LABEL2ID.items()}

_y = [LABEL2ID[data["label"]] for data in TRAIN_DATA]
# GT means Ground Truth
TRAIN_GT, DEV_GT = train_test_split(
    DOC_DATA,
    test_size=0.03,
    random_state=SEED,
    shuffle=True,
    stratify=_y,
)


Preload wiki database (1 min)

In [12]:
wiki_pages = jsonl_dir_to_df("data/wiki-pages")
mapping = generate_evidence_to_wiki_pages_mapping(wiki_pages)
del wiki_pages

Reading and concatenating jsonl files in data/wiki-pages
Generate parse mapping


Transform to id to evidence_map mapping


### Helper function

內含:

* 調整input樣子的code (是否要擴增資料集)

使用方式:

* 蓋起來直接執行

In [13]:
import json

def evidence_macro_precision(
    instance: Dict,
    top_rows: pd.DataFrame,
) -> Tuple[float, float]:
    """Calculate precision for sentence retrieval
    This function is modified from fever-scorer.
    https://github.com/sheffieldnlp/fever-scorer/blob/master/src/fever/scorer.py

    Args:
        instance (dict): a row of the dev set (dev.jsonl) of test set (test.jsonl)
        top_rows (pd.DataFrame): our predictions with the top probabilities

        IMPORTANT!!!
        instance (dict) should have the key of `evidence`.
        top_rows (pd.DataFrame) should have a column `predicted_evidence`.

    Returns:
        Tuple[float, float]:
        [1]: relevant and retrieved (numerator of precision)
        [2]: retrieved (denominator of precision)
    """
    this_precision = 0.0
    this_precision_hits = 0.0

    # Return 0, 0 if label is not enough info since not enough info does not
    # contain any evidence.
    if instance["label"].upper() != "NOT ENOUGH INFO":
        # e[2] is the page title, e[3] is the sentence index
        all_evi = [[e[2], e[3]]
                   for eg in instance["evidence"]
                   for e in eg
                   if e[3] is not None]
        claim = instance["claim"]
        predicted_evidence = top_rows[top_rows["claim"] ==
                                      claim]["predicted_evidence"].tolist()

        for prediction in predicted_evidence:
            if prediction in all_evi:
                this_precision += 1.0
            this_precision_hits += 1.0

        return (this_precision /
                this_precision_hits) if this_precision_hits > 0 else 1.0, 1.0

    return 0.0, 0.0

def evidence_macro_recall(
    instance: Dict,
    top_rows: pd.DataFrame,
) -> Tuple[float, float]:
    """Calculate recall for sentence retrieval
    This function is modified from fever-scorer.
    https://github.com/sheffieldnlp/fever-scorer/blob/master/src/fever/scorer.py

    Args:
        instance (dict): a row of the dev set (dev.jsonl) of test set (test.jsonl)
        top_rows (pd.DataFrame): our predictions with the top probabilities

        IMPORTANT!!!
        instance (dict) should have the key of `evidence`.
        top_rows (pd.DataFrame) should have a column `predicted_evidence`.

    Returns:
        Tuple[float, float]:
        [1]: relevant and retrieved (numerator of recall)
        [2]: relevant (denominator of recall)
    """
    # We only want to score F1/Precision/Recall of recalled evidence for NEI claims
    if instance["label"].upper() != "NOT ENOUGH INFO":
        # If there's no evidence to predict, return 1
        if len(instance["evidence"]) == 0 or all(
            [len(eg) == 0 for eg in instance]):
            return 1.0, 1.0

        claim = instance["claim"]

        predicted_evidence = top_rows[top_rows["claim"] ==
                                      claim]["predicted_evidence"].tolist()

        for evidence_group in instance["evidence"]:
            evidence = [[e[2], e[3]] for e in evidence_group]
            if all([item in predicted_evidence for item in evidence]):
                # We only want to score complete groups of evidence. Incomplete
                # groups are worthless.
                return 1.0, 1.0
        return 0.0, 1.0
    return 0.0, 0.0

def evaluate_retrieval(
    probs: np.ndarray,
    df_evidences: pd.DataFrame,
    ground_truths: pd.DataFrame,
    top_n: int = 5,
    cal_scores: bool = True,
    save_name: str = None,
    shut_up = False,
    probs_limit = 0.55
) -> Dict[str, float]:
    """Calculate the scores of sentence retrieval

    Args:
        probs (np.ndarray): probabilities of the candidate retrieved sentences
        df_evidences (pd.DataFrame): the candiate evidence sentences paired with claims
        ground_truths (pd.DataFrame): the loaded data of dev.jsonl or test.jsonl
        top_n (int, optional): the number of the retrieved sentences. Defaults to 2.

    Returns:
        Dict[str, float]: F1 score, precision, and recall
    """
    df_evidences["prob"] = probs
    top_rows = (
        df_evidences.groupby("claim").apply(
        lambda x: x.nlargest(top_n, "prob"))
        .reset_index(drop=True)
    )

    if shut_up==False:
      print("before filter top_rows shape : ",top_rows.shape)
      
    top_rows=top_rows[top_rows["prob"]>probs_limit]
    top_rows=top_rows.reset_index(drop=True)
      
    if shut_up==False:
      print("after filter top_rows shape : ",top_rows.shape)

    if shut_up==False:
      print("top_rows=")
      for i in range(30):
        print(top_rows["claim"][i],top_rows["text"][i])
        # print(top_rows["claim"][i],top_rows["text"][i],top_rows["evidence"][i])
        # for j in top_rows["evidence"][i]:

    if cal_scores:
        macro_precision = 0
        macro_precision_hits = 0
        macro_recall = 0
        macro_recall_hits = 0

        for i, instance in enumerate(ground_truths):
            macro_prec = evidence_macro_precision(instance, top_rows)
            macro_precision += macro_prec[0]
            macro_precision_hits += macro_prec[1]

            macro_rec = evidence_macro_recall(instance, top_rows)
            macro_recall += macro_rec[0]
            macro_recall_hits += macro_rec[1]

        pr = (macro_precision /
              macro_precision_hits) if macro_precision_hits > 0 else 1.0
        rec = (macro_recall /
               macro_recall_hits) if macro_recall_hits > 0 else 0.0
        f1 = 2.0 * pr * rec / (pr + rec)

    if save_name is not None:
        # write doc7_sent5 file
        with open(f"data/{save_name}", "w") as f:
            for instance in ground_truths:
                claim = instance["claim"]
                predicted_evidence = top_rows[
                    top_rows["claim"] == claim]["predicted_evidence"].tolist()
                instance["predicted_evidence"] = predicted_evidence
                f.write(json.dumps(instance, ensure_ascii=False) + "\n")

    if cal_scores:
        return {"F1 score": f1, "Precision": pr, "Recall": rec}

def get_predicted_probs(
    model: nn.Module,
    dataloader: Dataset,
    device: torch.device,
) -> np.ndarray:
    """Inference script to get probabilites for the candidate evidence sentences

    Args:
        model: the one from HuggingFace Transformers
        dataloader: devset or testset in torch dataloader

    Returns:
        np.ndarray: probabilites of the candidate evidence sentences
    """
    model.eval()
    probs = []

    with torch.no_grad():
        for batch in tqdm(dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits
            probs.extend(torch.softmax(logits, dim=1)[:, 1].tolist())

    return np.array(probs)

class SentRetrievalBERTDataset(BERTDataset):
    """SentRetrievalBERTDataset class for AICUP dataset with top-k evidence sentences."""

    def __getitem__(
        self,
        idx: int,
        **kwargs,
    ) -> Tuple[Dict[str, torch.Tensor], int]:
        item = self.data.iloc[idx]
        sentA = item["claim"]
        sentB = item["text"]

        # claim [SEP] text
        concat = self.tokenizer(
            sentA,
            sentB,
            padding="max_length",
            max_length=self.max_length,
            truncation=True,
        )
        concat_ten = {k: torch.tensor(v) for k, v in concat.items()}
        if "label" in item:
            concat_ten["labels"] = torch.tensor(item["label"])

        return concat_ten





In [14]:
import random

def pair_with_wiki_sentences(
    mapping: Dict[str, Dict[int, str]],
    df: pd.DataFrame,
    negative_ratio: float,
    comma  : str ,
    period  : str ,
    random_choose : str ,
    sent_seperate : str 
) -> pd.DataFrame:
    """Only for creating train sentences."""
    claims = []
    sentences = []
    labels = []


    # positive
    for i in range(len(df)):
        if df["label"].iloc[i] == "NOT ENOUGH INFO":
            continue

        claim = df["claim"].iloc[i]
        evidence_sets = df["evidence"].iloc[i]
        for evidence_set in evidence_sets:
            sents = []
            for evidence in evidence_set:
                # evidence[2] is the page title
                page = evidence[2].replace(" ", "_")
                # the only page with weird name
                if page == "臺灣海峽危機#第二次臺灣海峽危機（1958）":
                    continue
                # evidence[3] is in form of int however, mapping requires str
                sent_idx = str(evidence[3])
                
                #sents.append(mapping[page][sent_idx])

                if mapping[page][sent_idx]!="":

                  if sent_seperate == "yes" :
                    claims.append(claim)
                    sentences.append(mapping[page][sent_idx]) ### 拆句子
                    labels.append(1)
                    sents.append(mapping[page][sent_idx])

                  # 用,分隔 (做出更多dataset)

                  if comma == "yes" :
                    for i in mapping[page][sent_idx].split(","): 
                      claims.append(claim)
                      sentences.append(i) ### 所有相關句合再一起
                      labels.append(1)

                  if period == "yes" :
                    for i in mapping[page][sent_idx].split("。"):
                      claims.append(claim)
                      sentences.append(i) ### 所有相關句合再一起
                      labels.append(1)

            ### Random 合成句子 (做出更多dataset)
            if random_choose == "yes" :
              escape_rate=0.2
              dataset_set=set()

              while True:
                if random.random() < escape_rate:
                  break
                
                composed_sentence=[]
                

                for sub_sentence in sents:
                  if random.random() >=0.5:
                    composed_sentence.append(sub_sentence)
                
                if len(composed_sentence)!=0:
                  try_sentence="".join(composed_sentence)
                  if try_sentence not in dataset_set:
                    dataset_set.add(try_sentence)
                    # random.shuffle(composed_sentence)
                    # target_sentence="".join()
                    claims.append(claim)
                    sentences.append(try_sentence) ### 所有相關句合再一起
                    labels.append(1)

                escape_rate=escape_rate + random.random()*0.08

                

            #whole_evidence = " ".join(sents)

            # claims.append(claim)
            # sentences.append(whole_evidence) ### 所有相關句合再一起
            # labels.append(1)

    # negative
    for i in range(len(df)):
        if df["label"].iloc[i] == "NOT ENOUGH INFO":
            continue
        claim = df["claim"].iloc[i]

        evidence_set = set([(evidence[2], evidence[3])
                            for evidences in df["evidence"][i]
                            for evidence in evidences])
        predicted_pages = df["predicted_pages"][i]
        for page in predicted_pages:
            page = page.replace(" ", "_")
            try:
                page_sent_id_pairs = [
                    (page, sent_idx) for sent_idx in mapping[page].keys()
                ]
            except KeyError:
                # print(f"{page} is not in our Wiki db.")
                continue

            for pair in page_sent_id_pairs:
                if pair in evidence_set:
                    continue
                text = mapping[page][pair[1]]
                # `np.random.rand(1) <= 0.05`: Control not to add too many negative samples
                if text != "" and np.random.rand(1) <= negative_ratio:
                    claims.append(claim)
                    sentences.append(text)
                    labels.append(0)

    return pd.DataFrame({"claim": claims, "text": sentences, "label": labels})


def pair_with_wiki_sentences_eval(
    mapping: Dict[str, Dict[int, str]],
    df: pd.DataFrame,
    is_testset: bool = False,
) -> pd.DataFrame:
    """Only for creating dev and test sentences."""
    claims = []
    sentences = []
    evidence = []
    predicted_evidence = []

    # negative
    for i in range(len(df)):
        # if df["label"].iloc[i] == "NOT ENOUGH INFO":
        #     continue
        claim = df["claim"].iloc[i]

        predicted_pages = df["predicted_pages"][i]
        for page in predicted_pages:
            page = page.replace(" ", "_")
            try:
                page_sent_id_pairs = [(page, k) for k in mapping[page]]
            except KeyError:
                # print(f"{page} is not in our Wiki db.")
                continue

            for page_name, sentence_id in page_sent_id_pairs:
                text = mapping[page][sentence_id]
                if text != "":
                    claims.append(claim)
                    sentences.append(text)
                    if not is_testset:
                        evidence.append(df["evidence"].iloc[i])
                    predicted_evidence.append([page_name, int(sentence_id)])

    return pd.DataFrame({
        "claim": claims,
        "text": sentences,
        "evidence": evidence if not is_testset else None,
        "predicted_evidence": predicted_evidence,
    })

In [15]:
def pair_with_wiki_sentences_by_jerry(
    mapping: Dict[str, Dict[int, str]],
    df: pd.DataFrame,
    negative_ratio: float,
) -> pd.DataFrame:
    """Only for creating train sentences."""
    claims = []
    sentences = []
    labels = []

    # positive
    for i in range(len(df)):
        if df["label"].iloc[i] == "NOT ENOUGH INFO":
            continue

        claim = df["claim"].iloc[i]
        evidence_sets = df["evidence"].iloc[i]
        for evidence_set in evidence_sets:
            sents = []
            for evidence in evidence_set:
                # evidence[2] is the page title
                page = evidence[2].replace(" ", "_")
                # the only page with weird name
                if page == "臺灣海峽危機#第二次臺灣海峽危機（1958）":
                    continue
                # evidence[3] is in form of int however, mapping requires str
                sent_idx = str(evidence[3])
                sents.append(mapping[page][sent_idx])

            whole_evidence = " ".join(sents)

            claims.append(claim)
            sentences.append(whole_evidence)
            labels.append(1)

    # negative
    for i in range(len(df)):
        if df["label"].iloc[i] == "NOT ENOUGH INFO":
            continue
        claim = df["claim"].iloc[i]

        evidence_set = set([(evidence[2], evidence[3])
                            for evidences in df["evidence"][i]
                            for evidence in evidences])
        predicted_pages = df["predicted_pages"][i]
        for page in predicted_pages:
            page = page.replace(" ", "_")
            try:
                page_sent_id_pairs = [
                    (page, sent_idx) for sent_idx in mapping[page].keys()
                ]
            except KeyError:
                # print(f"{page} is not in our Wiki db.")
                continue

            for pair in page_sent_id_pairs:
                if pair in evidence_set:
                    continue
                text = mapping[page][pair[1]]
                # `np.random.rand(1) <= 0.05`: Control not to add too many negative samples
                if text != "" and np.random.rand(1) <= negative_ratio:
                    claims.append(claim)
                    sentences.append(text)
                    labels.append(0)

    return pd.DataFrame({"claim": claims, "text": sentences, "label": labels})


def pair_with_wiki_sentences_eval(
    mapping: Dict[str, Dict[int, str]],
    df: pd.DataFrame,
    is_testset: bool = False,
) -> pd.DataFrame:
    """Only for creating dev and test sentences."""
    claims = []
    sentences = []
    evidence = []
    predicted_evidence = []

    # negative
    for i in range(len(df)):
        # if df["label"].iloc[i] == "NOT ENOUGH INFO":
        #     continue
        claim = df["claim"].iloc[i]

        predicted_pages = df["predicted_pages"][i]
        for page in predicted_pages:
            page = page.replace(" ", "_")
            try:
                page_sent_id_pairs = [(page, k) for k in mapping[page]]
            except KeyError:
                # print(f"{page} is not in our Wiki db.")
                continue

            for page_name, sentence_id in page_sent_id_pairs:
                text = mapping[page][sentence_id]
                if text != "":
                    claims.append(claim)
                    sentences.append(text)
                    if not is_testset:
                        evidence.append(df["evidence"].iloc[i])
                    predicted_evidence.append([page_name, int(sentence_id)])

    return pd.DataFrame({
        "claim": claims,
        "text": sentences,
        "evidence": evidence if not is_testset else None,
        "predicted_evidence": predicted_evidence,
    })

### Part2-1. Train By Bert : 超參數


Hyperparams

In [16]:
MODEL_NAME = "bert-base-chinese"  #@param {type:"string"}
NUM_EPOCHS = 20  #@param {type:"integer"}
LR = 1e-5  #@param {type:"number"}
TRAIN_BATCH_SIZE = 64  #@param {type:"integer"}
TEST_BATCH_SIZE = 64  #@param {type:"integer"}


TOP_N = 5  #@param {type:"integer"}

### 填入yes(要擴)/no
SENT_SEPERATE = "yes" #@param {type:"string"}
COMMA   = "yes" #@param {type:"string"}
PERIOD  = "no" #@param {type:"string"}
RANDOM_CHOOSE  = "yes" #@param {type:"string"}

PROB_LIMIT =  0.75  #@param {type:"number"}

### Run Faster ([IMPORTANT] 真正Train的時候要為no，不然會白Train)
Testing_or_not = "no" #@param {type:"string"}


In [17]:
NEGATIVE_RATIO = 0.1096  #@param {type:"number"}

if SENT_SEPERATE == "yes" or COMMA == "yes" or PERIOD == "yes" or RANDOM_CHOOSE == "yes": ### 要資料擴充的跑這邊 (會比較久)
  train_df = pair_with_wiki_sentences(
      mapping,
      pd.DataFrame(TRAIN_GT),
      NEGATIVE_RATIO,
      comma  = COMMA,
      period  = PERIOD,
      random_choose = RANDOM_CHOOSE,
      sent_seperate = SENT_SEPERATE
  )
  counts = train_df["label"].value_counts()
  print("Now using the following train data with 0 (Negative) and 1 (Positive)")
  print(counts)

  dev_evidences = pair_with_wiki_sentences_eval(mapping, pd.DataFrame(DEV_GT))

else:   ### 不要資料擴充的跑這邊
  train_df = pair_with_wiki_sentences_by_jerry(
      mapping,
      pd.DataFrame(TRAIN_GT),
      NEGATIVE_RATIO,
  )
  counts = train_df["label"].value_counts()
  print("Now using the following train data with 0 (Negative) and 1 (Positive)")
  print(counts)

  dev_evidences = pair_with_wiki_sentences_eval(mapping, pd.DataFrame(DEV_GT))

if Testing_or_not=="yes":
  DEV_GT=DEV_GT[:100]
  dev_evidences = pair_with_wiki_sentences_eval(mapping, pd.DataFrame(DEV_GT))

print("DEV_GT lens : ",len(DEV_GT))

### 請確定0,1比例正常 (by 調整Negative Rate)

Now using the following train data with 0 (Negative) and 1 (Positive)
0    34046
1    33977
Name: label, dtype: int64
DEV_GT lens :  349


In [18]:
EXP_DIR = f"sent_retrieval/e{NUM_EPOCHS}_bs{TRAIN_BATCH_SIZE}_" + f"{LR}_neg{NEGATIVE_RATIO}_top{TOP_N}_data{Use_Data_From_Part_1}_prob{PROB_LIMIT}_{SENT_SEPERATE}_{COMMA}_{PERIOD}_{RANDOM_CHOOSE}"
LOG_DIR = "logs/" + EXP_DIR
CKPT_DIR = "checkpoints/" + EXP_DIR

HANG_MODEL_NAME= f"e{NUM_EPOCHS}_bs{TRAIN_BATCH_SIZE}_" + f"{LR}_neg{NEGATIVE_RATIO}_top{TOP_N}"

if not Path(LOG_DIR).exists():
    Path(LOG_DIR).mkdir(parents=True)

if not Path(CKPT_DIR).exists():
    Path(CKPT_DIR).mkdir(parents=True)

print("儲存路徑 : ",EXP_DIR)
print("[IMPORTANT] record to excel")

儲存路徑 :  sent_retrieval/e20_bs64_1e-05_neg0.1096_top5_data4_prob0.75_yes_yes_no_yes
[IMPORTANT] record to excel


### Part2-2.Train By Bert : 前置

蓋上直接執行

Dataloader things. Please refer to [PyTorch Dataset tutorial](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)


In [19]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_dataset = SentRetrievalBERTDataset(train_df, tokenizer=tokenizer)
val_dataset = SentRetrievalBERTDataset(dev_evidences, tokenizer=tokenizer)

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=TRAIN_BATCH_SIZE,
)
eval_dataloader = DataLoader(val_dataset, batch_size=TEST_BATCH_SIZE)

Save your memory.

In [20]:
del train_df

Trainer

In [21]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device(
    "cpu")
print("Use Device : ",device)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
model.to(device)

optimizer = AdamW(model.parameters(), lr=LR)
num_training_steps = NUM_EPOCHS * len(train_dataloader)
lr_scheduler = set_lr_scheduler(optimizer, num_training_steps)

writer = SummaryWriter(LOG_DIR)

Use Device :  cuda


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

For training, please refer to [PyTorch Optimization tutorial](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html)

Please make sure that you are using gpu when training (5 min)

### Part2-3-1.Train By Bert : Training By Interrupted Model

可能因為被中斷要接續Train

In [71]:
print("CKPT_DIR=",CKPT_DIR)

CKPT_DIR= checkpoints/sent_retrieval/e20_bs64_1e-05_neg0.115_top5_data4_prob0.75_yes_yes_no_yes


In [72]:
### 可以改動選Model的資料夾 ### Like checkpoints/sent_retrieval/資料夾名

# CKPT_DIR = "checkpoints/sent_retrieval/e20_bs64_2e-05_neg0.12_top5"  #@param {type:"string"}

In [102]:
### 如果是中途插入Model (不Train) ，要先跑建立Model的Code


# MODEL_NAME = "bert-base-chinese"

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
# model.to(device)


In [73]:
ls checkpoints/sent_retrieval/e20_bs64_1e-05_neg0.115_top5_data4_prob0.75_yes_yes_no_yes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
 F1_0.6095431119224178_Precision0.5030786201363847_model.890.pt
 F1_0.6540758268388019_Precision0.5580525471319716_model.1780.pt
 F1_0.7160320102627252_Precision0.6852587244284006_model.2670.pt
 F1_0.7210003266626941_Precision0.6829522663457691_model.5340.pt
 F1_0.737284387872533_Precision0.7035900521460097_model.890.pt
 F1_0.7470057503933136_Precision0.7300742077817898_model.890.pt
 F1_0.754868552205756_Precision0.7546229442438831_model.1780.pt
 F1_0.7552447136792517_Precision0.7571901323706383_model.890.pt
 F1_0.7557266634269273_Precision0.7575511432009626_model.1780.pt
 F1_0.7573400417066631_Precision0.7638888888888888_model.2670.pt
'val_acc=0.4819_model.380.pt'
'v

In [104]:
# # #選被中斷的Trainning Model


# ckpt_name = "model.1800.pt"  #@param {type:"string"}
# model = load_model(model, ckpt_name, CKPT_DIR)


### Part2-3-2.Train By Bert : Training

每隔Validate會在/NCKU/checkpoints/Sent裡面放入模型

In [20]:
### 選 Validation Step
print("it has ",len(train_dataloader)," batchs in an epoch")

it has  1060  batchs in an epoch


In [21]:
### 選 Validation Step (和len(dataloader)差不多 == 1 epoch　val 一次)

VALIDATION_STEP = 500  #@param {type:"integer"}

In [22]:
best_v_recall = 0.0 #@param {type:"number"}
best_v_f1 = 0.0 #@param {type:"number"}
best_v_precision = 0.0 #@param {type:"number"}


In [23]:
progress_bar = tqdm(range(num_training_steps))
current_steps = 0


for epoch in range(NUM_EPOCHS):
    model.train()

    print("[Epoch {:d}/{:d}]".format(epoch, NUM_EPOCHS))
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        writer.add_scalar("training_loss", loss.item(), current_steps)

        y_pred = torch.argmax(outputs.logits, dim=1).tolist()
        y_true = batch["labels"].tolist()

        current_steps += 1

        if current_steps % VALIDATION_STEP == 0 and current_steps > 0:
            print("Start validation")
            probs = get_predicted_probs(model, eval_dataloader, device)

            val_results = evaluate_retrieval(
                probs=probs,
                df_evidences=dev_evidences,
                ground_truths=DEV_GT,
                top_n=TOP_N,
                shut_up=True, ### 如果不想讓他輸出很多 shut_up=True
                probs_limit = 0.55
            )
            print(val_results)

            # log each metric separately to TensorBoard
            for metric_name, metric_value in val_results.items():
                writer.add_scalar(
                    f"dev_{metric_name}",
                    metric_value,
                    current_steps,
                )

            name_p=int(best_v_precision*100)
            name_f=int(best_v_f1*100)

            if val_results["Precision"]>best_v_precision:
              best_v_precision=val_results["Precision"]

              save_checkpoint(model, CKPT_DIR, current_steps,mark="F1_"+str(val_results["F1 score"])+"_Precision"+str(val_results["Precision"]))
              print("[New Record of Precision] at ",current_steps)
              print("model saved at ",CKPT_DIR)

            if val_results["F1 score"]>best_v_f1:
              best_v_f1=val_results["F1 score"]
              save_checkpoint(model, CKPT_DIR, current_steps,mark="F1_"+str(val_results["F1 score"])+"_Precision"+str(val_results["Precision"]))
              print("[New Record of F1] at ",current_steps)
              print("model saved at ",CKPT_DIR)

print("Finished training!")
print("[IMPORTANT] record model name to excel")

  0%|          | 0/21200 [00:00<?, ?it/s]

[Epoch 0/20]
Start validation


  0%|          | 0/203 [00:00<?, ?it/s]

{'F1 score': 0.5892494648443934, 'Precision': 0.5026000000000004, 'Recall': 0.712}
[New Record of Precision] at  500
model saved at  checkpoints/sent_retrieval/e20_bs64_1e-05_neg0.1096_top5_data4_prob0.75_yes_yes_no_yes
[New Record of F1] at  500
model saved at  checkpoints/sent_retrieval/e20_bs64_1e-05_neg0.1096_top5_data4_prob0.75_yes_yes_no_yes
Start validation


  0%|          | 0/203 [00:00<?, ?it/s]

...

  0%|          | 0/203 [00:00<?, ?it/s]

{'F1 score': 0.6851222853811396, 'Precision': 0.6601999999999999, 'Recall': 0.712}
Start validation


  0%|          | 0/203 [00:00<?, ?it/s]

{'F1 score': 0.6797889594528579, 'Precision': 0.6406666666666666, 'Recall': 0.724}


KeyboardInterrupt: 

### Part2-4.Train By Bert : 統整Acc與幫資料存檔

除非要生成給Part3 的Train與Validate，不然不要執行

可以先生成Model再1-4(兩階段執行，但是要執行Part0 & Part2始到1-1前的Code)


In [55]:
### 放上來前記得要包含F1("F"+F1值) /Precision ("P"+Precision值)

print("CKPT_DIR=",CKPT_DIR)

CKPT_DIR= checkpoints/sent_retrieval/e20_bs64_1e-05_neg0.078_top5_data3_prob0.55_yes_no_yes_yes


In [ ]:
### 可以改動選Model的資料夾 ### Like checkpoints/sent_retrieval/資料夾名

# CKPT_DIR = "checkpoints/sent_retrieval/e20_bs64_1e-05_neg0.057_top5_data3_yes"  #@param {type:"string"}


In [58]:
### 決定給Part3的檔案名稱

MODEL_PARA=CKPT_DIR.split("/")
MODEL_PARA=MODEL_PARA[-1]
print("Model 參數: ",MODEL_PARA," [填入下方]")


### 如果是中途插入，要填入參數

# TOP_N = 5  #@param {type:"integer"}
# Use_Data_From_Part_1 = "3"  #@param {type:"string"}

###


print("儲存檔名(Train) : ",f"train_doc5sent{TOP_N}_data{Use_Data_From_Part_1}_{MODEL_PARA}.jsonl")
print("儲存檔名(Valid) : ",f"dev_doc5sent{TOP_N}_data{Use_Data_From_Part_1}_{MODEL_PARA}.jsonl")

Model 參數:  e20_bs64_1e-05_neg0.078_top5_data3_prob0.55_yes_no_yes_yes  [填入下方]
儲存檔名(Train) :  train_doc5sent5_data3_e20_bs64_1e-05_neg0.078_top5_data3_prob0.55_yes_no_yes_yes.jsonl
儲存檔名(Valid) :  dev_doc5sent5_data3_e20_bs64_1e-05_neg0.078_top5_data3_prob0.55_yes_no_yes_yes.jsonl


In [ ]:
## 如果是中途插入Model (不Train) ，要先跑建立Model的Code


# MODEL_NAME = "bert-base-chinese"

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# print("使用裝置 : ",device)
# model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
# model.to(device)


In [57]:
!ls checkpoints/sent_retrieval/e20_bs64_1e-05_neg0.078_top5_data3_prob0.55_yes_no_yes_yes

F1_0.3146156909597769_Precision0.21944444444444433_model.3.pt
F1_0.32255244755244744_Precision0.2249999999999999_model.6.pt


In [ ]:
#選Validate Model


ckpt_name = "model.1800.pt"  #@param {type:"string"}
model = load_model(model, ckpt_name, CKPT_DIR)
print("Start final evaluations and write prediction files.")



Start final evaluations and write prediction files.


In [ ]:

import json

train_evidences = pair_with_wiki_sentences_eval(
    mapping=mapping,
    df=pd.DataFrame(TRAIN_GT),
)
train_set = SentRetrievalBERTDataset(train_evidences, tokenizer)
train_dataloader = DataLoader(train_set, batch_size=TEST_BATCH_SIZE)

print("Start calculating training scores")
probs = get_predicted_probs(model, train_dataloader, device)
train_results = evaluate_retrieval(
    probs=probs,
    df_evidences=train_evidences,
    ground_truths=TRAIN_GT,
    top_n=TOP_N,
    save_name=f"train_doc5sent{TOP_N}_data{Use_Data_From_Part_1}_{MODEL_PARA}.jsonl",
    shut_up=True
)
print(f"Training scores => {train_results}")

dev_evidences = pair_with_wiki_sentences_eval(mapping, pd.DataFrame(DEV_GT))

val_dataset = SentRetrievalBERTDataset(dev_evidences, tokenizer=tokenizer)

eval_dataloader = DataLoader(val_dataset, batch_size=TEST_BATCH_SIZE)


print("Start validation")
probs = get_predicted_probs(model, eval_dataloader, device)
val_results = evaluate_retrieval(
    probs=probs,
    df_evidences=dev_evidences,
    ground_truths=DEV_GT,
    top_n=TOP_N,
    save_name=f"dev_doc5sent{TOP_N}_data{Use_Data_From_Part_1}_{MODEL_PARA}.jsonl",
    shut_up=True
)

print(f"Validation scores => {val_results}")

Start calculating training scores


  0%|          | 0/10481 [00:00<?, ?it/s]

Training scores => {'F1 score': 0.43861306440105235, 'Precision': 0.2982242287434378, 'Recall': 0.8287434161023326}
Start validation


  0%|          | 0/2636 [00:00<?, ?it/s]

Validation scores => {'F1 score': 0.4276453672723141, 'Precision': 0.28915964701162716, 'Recall': 0.8206979542719615}


### Part2-5.Train By Bert : Testing and Make File

In [22]:
import json

In [23]:
! ls data | grep test

hanlp_con_test_results.pkl
private+public_test_data.jsonl
private_test_data.jsonl
public_test.jsonl
test_doc5_data1.jsonl
test_doc5_data3.jsonl
test_doc5_data4.jsonl
test_doc5sent5_data1_e20_bs32_7e-05_top5.jsonl
test_doc5sent5_data1_e20_bs32_7e-05_top5.pkl
test_doc5sent5_data1_e20_bs64_1e-05_neg0.232_top5_data1_yes.jsonl
test_doc5sent5_data1_e20_bs64_1e-05_neg0.232_top5_data1_yes.pkl
test_doc5sent5_data3_e20_bs64_1e-05_neg0.057_top5_data3_yes.jsonl
test_doc5sent5_data3_e20_bs64_1e-05_neg0.057_top5_data3_yes.pkl
test_doc5sent5_data4_e20_bs64_1e-05_neg0.0665_top5_data4_yes.jsonl
test_doc5sent5_data4_e20_bs64_1e-05_neg0.0665_top5_data4_yes.pkl
test_doc5sent5_data4_e20_bs64_1e-05_neg0.115_top5_data4_prob0.75_yes_yes_no_yes.jsonl
test_doc5sent5_data4_e20_bs64_1e-05_neg0.115_top5_data4_prob0.75_yes_yes_no_yes.pkl


In [25]:
### 選Part 1 執行結果的檔案
### 改右邊就好
### 不要/data

Data_From_Part_1_Test = ""  #@param {type:"string"}

file_name= f"test_doc5_data{Data_From_Part_1_Test}.jsonl"

print("Use file ",file_name)

test_data = load_json("data/"+file_name)

Use file  test_doc5_data4.jsonl


In [ ]:
# 如果Model路徑不慎滿意可以調

DIR_FOR_MODEL="checkpoints/sent_retrieval/e20_bs64_1e-05_neg0.057_top5_data3_yes" #@param {type:"string"}
CKPT_DIR = DIR_FOR_MODEL

# 並配合 LS 服用

!ls "checkpoints/sent_retrieval/e20_bs64_1e-05_neg0.057_top5_data3_yes"

### 決定給Part3的檔案名稱

MODEL_PARA=CKPT_DIR.split("/")
MODEL_PARA=MODEL_PARA[-1]
print("Model 參數: ",MODEL_PARA)


### 如果是中途插入，要填入參數

# TOP_N = 5  #@param {type:"integer"}
# Use_Data_From_Part_1 = "3"  #@param {type:"string"}

###


print("儲存檔名(Test) : ",f"test_doc5sent{TOP_N}_data{Data_From_Part_1_Test}_{MODEL_PARA}.jsonl")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
model.1000.pt  model.1600.pt  model.250.pt  model.600.pt
model.1200.pt  model.1800.pt  model.500.pt
Model 參數:  e20_bs64_1e-05_neg0.057_top5_data3_yes
儲存檔名(Test) :  test_doc5sent5_data3_e20_bs64_1e-05_neg0.057_top5_data3_yes.jsonl


In [ ]:
## 如果是中途插入Model (不Train) ，要先跑建立Model的Code


# MODEL_NAME = "bert-base-chinese"

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# print("使用裝置 : ",device)
# model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
# model.to(device)


In [ ]:
### 選擇Model

ckpt_name = "model.1800.pt"  #@param {type:"string"}
model = load_model(model, ckpt_name, CKPT_DIR)


In [ ]:
import json


test_evidences = pair_with_wiki_sentences_eval(
    mapping,
    pd.DataFrame(test_data),
    is_testset=True,
)
test_set = SentRetrievalBERTDataset(test_evidences, tokenizer)
test_dataloader = DataLoader(test_set, batch_size=TEST_BATCH_SIZE)

print("Start predicting the test data")
probs = get_predicted_probs(model, test_dataloader, device)
evaluate_retrieval(
    probs=probs,
    df_evidences=test_evidences,
    ground_truths=test_data,
    top_n=TOP_N,
    cal_scores=False,
    save_name=f"test_doc5sent{TOP_N}_data{Data_From_Part_1_Test}_{MODEL_PARA}.jsonl",
    shut_up=True
)

Start predicting the test data


  0%|          | 0/960 [00:00<?, ?it/s]

In [ ]:
print("Saved name : ",f"test_doc5sent{TOP_N}_data{Data_From_Part_1_Test}_{MODEL_PARA}.jsonl")

Saved name :  test_doc5sent5_data3_e20_bs64_1e-05_neg0.057_top5_data3_yes.jsonl


notebook3
## PART 3. Claim verification
難度：★★☆☆☆

### 參數集中輸入

In [36]:
### 用幾號Data
Use_Data_From_Part_1 = "4"  #@param {type:"string"}

### Part2 輸出至多多少句子(通常5)
Par2_Number_of_Sent = "5"  #@param {type:"string"}


### 用哪一個Part2 Model (填路徑) (ex checkpoints/sent_retrieval/xxxxx)
Part2_Model_Path = "checkpoints/sent_retrieval/e20_bs64_1e-05_neg0.115_top5_data4_prob0.75_yes_yes_no_yes"  #@param {type:"string"}
print("[IMPORTANT] record Part2 Path to excel")

### 要幾個<SEP>
SEP_NUMBER = 0 #@param {type:"integer"}

# 配合 LS 服用，幫你確定沒有找錯資料夾
### [這個要記得改]
!ls "checkpoints/sent_retrieval/e20_bs64_1e-05_neg0.078_top5_data3_prob0.55_yes_no_yes_yes"

[IMPORTANT] record Part2 Path to excel
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
F1_0.6357835191600284_Precision0.5452667468912974_model.2300.pt


In [37]:

CKPT_DIR = Part2_Model_Path


### 決定給Part3的檔案名稱

MODEL_PARA=CKPT_DIR.split("/")
MODEL_PARA=MODEL_PARA[-1]
print("Model 參數: ",MODEL_PARA)

### 檢查


print("使用檔名(Train) : ",f"data/train_doc5sent{Par2_Number_of_Sent}_data{Use_Data_From_Part_1}_{MODEL_PARA}.jsonl")
print("使用檔名(Valid) : ",f"data/dev_doc5sent{Par2_Number_of_Sent}_data{Use_Data_From_Part_1}_{MODEL_PARA}.jsonl")

Model 參數:  e20_bs64_1e-05_neg0.115_top5_data4_prob0.75_yes_yes_no_yes
使用檔名(Train) :  data/train_doc5sent5_data4_e20_bs64_1e-05_neg0.115_top5_data4_prob0.75_yes_yes_no_yes.jsonl
使用檔名(Valid) :  data/dev_doc5sent5_data4_e20_bs64_1e-05_neg0.115_top5_data4_prob0.75_yes_yes_no_yes.jsonl


In [38]:
! ls data | grep pkl

### Check PKL有沒有重複 (有的話不會重跑)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
dev_doc5sent5_data1_e20_bs64_1e-05_neg0.182_top5_data1_prob0.55_yes_no_yes_yes.pkl
dev_doc5sent5_data1_e20_bs64_1e-05_neg0.232_top5_data1_yes.pkl
dev_doc5sent5_data3_e20_bs64_1e-05_neg0.057_top5_data3_yes.pkl
dev_doc5sent5_data4_e20_bs64_1e-05_neg0.0665_top5_data4_yes.pkl
dev_doc5sent5_data4_e20_bs64_1e-05_neg0.115_top5_data4_prob0.75_yes_yes_no_yes.pkl
hanlp_con_results.pkl
hanlp_con_test_results.pkl
test_doc5sent5_data1_e20_bs32_7e-05_top5.pkl
test_doc5sent5_data1_e20_bs64_1e-05_neg0.232_top5_data1_yes.pkl
test_doc5sent5_data3_e20_bs64_1e-05_neg0.057_top5_data3_yes.pkl
test_doc5sent5_data4_e20_bs64_1e-05_neg0.0665_top5_data4_yes.pkl
train_doc5sent5_data1_e20_bs64_1e

### 選擇Training Data (依照Part1 生成的1,2,3)

蓋起來直接執行

In [39]:
import pickle
from pathlib import Path
from typing import Dict, Tuple

import numpy as np
import pandas as pd
from pandarallel import pandarallel
from tqdm.auto import tqdm

import torch
from sklearn.metrics import accuracy_score
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_scheduler,
)

from dataset import BERTDataset


pandarallel.initialize(progress_bar=True, verbose=0, nb_workers=4)

In [40]:
# local libs
from utils import (
    generate_evidence_to_wiki_pages_mapping,
    jsonl_dir_to_df,
    load_json,
    load_model,
    save_checkpoint,
    set_lr_scheduler,
)

In [41]:
### Trainning Data From Part1

### always in form : data/train_doc5_data{x}.jsonl

Train_File_From_Part_2 = f"data/train_doc5sent{Par2_Number_of_Sent}_data{Use_Data_From_Part_1}_{MODEL_PARA}.jsonl"  
Valid_File_From_Part_2 = f"data/dev_doc5sent{Par2_Number_of_Sent}_data{Use_Data_From_Part_1}_{MODEL_PARA}.jsonl" 

Train_Pkl_From_Part_2 = f"data/train_doc5sent{Par2_Number_of_Sent}_data{Use_Data_From_Part_1}_{MODEL_PARA}.pkl"
Valid_Pkl_From_Part_2 = f"data/dev_doc5sent{Par2_Number_of_Sent}_data{Use_Data_From_Part_1}_{MODEL_PARA}.pkl"

TRAIN_DATA = load_json(Train_File_From_Part_2)
DEV_DATA = load_json(Valid_File_From_Part_2)

TRAIN_PKL_FILE = Path(Train_Pkl_From_Part_2)
DEV_PKL_FILE = Path(Valid_Pkl_From_Part_2)

## 缺檔案時報錯

### 前置

內含wiki轉換(3min)

蓋起來直接執行

In [42]:
SEED = 42

LABEL2ID: Dict[str, int] = {
    "supports": 0,
    "refutes": 1,
    "NOT ENOUGH INFO": 2,
}
ID2LABEL: Dict[int, str] = {v: k for k, v in LABEL2ID.items()}



In [43]:
wiki_pages = jsonl_dir_to_df("data/wiki-pages")
mapping = generate_evidence_to_wiki_pages_mapping(wiki_pages,)
del wiki_pages

### mapping : {
  #  "page_name":{
      # "第n句":"文字"
  #  } 
###}

Reading and concatenating jsonl files in data/wiki-pages
Generate parse mapping


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

### Helper function

內可選資料集擴充與否

直接執行

In [44]:
class AicupTopkEvidenceBERTDataset(BERTDataset):
    """AICUP dataset with top-k evidence sentences."""

    def __getitem__(
        self,
        idx: int,
        **kwargs,
    ) -> Tuple[Dict[str, torch.Tensor], int]:
        item = self.data.iloc[idx]
        claim = item["claim"]
        evidence = item["evidence_list"]

        # In case there are less than topk evidence sentences
        pad = ["[PAD]"] * (self.topk - len(evidence))  ### 補齊For Trainning data?
        if type(evidence)==str:
          evidence=[evidence]

        evidence += pad
        # concat_claim_evidence = " [SEP] ".join([claim,"[SEP]","[SEP]", *evidence])
        concat_claim_evidence = " [SEP] ".join([claim, "[SEP]"*SEP_NUMBER ,*evidence])

        # if idx==2:
        #   print(concat_claim_evidence)

        concat = self.tokenizer(
            concat_claim_evidence,
            padding="max_length",
            max_length=self.max_length,
            truncation=True,
        )
        label = LABEL2ID[item["label"]] if "label" in item else -1
        concat_ten = {k: torch.tensor(v) for k, v in concat.items()}

        if "label" in item:
            concat_ten["labels"] = torch.tensor(label)

        return concat_ten

Evaluation function

In [45]:
def run_evaluation(model: torch.nn.Module, dataloader: DataLoader, device):
    model.eval()

    loss = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for batch in tqdm(dataloader):
            y_true.extend(batch["labels"].tolist())

            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss += outputs.loss.item()
            logits = outputs.logits
            y_pred.extend(torch.argmax(logits, dim=1).tolist())

    acc = accuracy_score(y_true, y_pred)

    return {"val_loss": loss / len(dataloader), "val_acc": acc}

Prediction

In [46]:
def run_predict(model: torch.nn.Module, test_dl: DataLoader, device) -> list:
    model.eval()

    preds = []
    for batch in tqdm(test_dl,
                      total=len(test_dl),
                      leave=False,
                      desc="Predicting"):
        batch = {k: v.to(device) for k, v in batch.items()}
        pred = model(**batch).logits
        pred = torch.argmax(pred, dim=1)
        preds.extend(pred.tolist())
    return preds

In [47]:
def join_with_topk_evidence(
    df: pd.DataFrame,
    mapping: dict,
    mode: str = "train",
    topk: int = 5,
) -> pd.DataFrame:
    """join_with_topk_evidence join the dataset with topk evidence.

    Note:
        After extraction, the dataset will be like this:
               id     label         claim                           evidence            evidence_list
        0    4604  supports       高行健...     [[[3393, 3552, 高行健, 0], [...  [高行健 （ ）江西赣州出...
        ..    ...       ...            ...                                ...                     ...
        945  2095  supports       美國總...  [[[1879, 2032, 吉米·卡特, 16], [...  [卸任后 ， 卡特積極參與...
        停各种战争及人質危機的斡旋工作 ， 反对美国小布什政府攻打伊拉克...

        [946 rows x 5 columns]

    Args:
        df (pd.DataFrame): The dataset with evidence.
        wiki_pages (pd.DataFrame): The wiki pages dataframe
        topk (int, optional): The topk evidence. Defaults to 5.
        cache(Union[Path, str], optional): The cache file path. Defaults to None.
            If cache is None, return the result directly.

    Returns:
        pd.DataFrame: The dataset with topk evidence_list.
            The `evidence_list` column will be: List[str]
    """

    # format evidence column to List[List[Tuple[str, str, str, str]]]
    if "evidence" in df.columns:
        df["evidence"] = df["evidence"].parallel_map(
            lambda x: [[x]] if not isinstance(x[0], list) else [x]
            if not isinstance(x[0][0], list) else x) ### to 3層list

    print(f"Extracting evidence_list for the {mode} mode ...")
    if mode == "eval":
        # extract evidence
        df["evidence_list"] = df["predicted_evidence"].parallel_map(lambda x: " ".join([
            mapping.get(evi_id, {}).get(str(evi_idx), "")
            for evi_id, evi_idx in x  # for each evidence list
        ][:topk]) if isinstance(x, list) else "")  ### 做出[前topk句]
        print(df["evidence_list"][:5])
    else:
        # extract evidence
        df["evidence_list"] = df["evidence"].parallel_map(lambda x: [
            " ".join([  # join evidence
                mapping.get(evi_id, {}).get(str(evi_idx), "")
                for _, _, evi_id, evi_idx in evi_list
            ]) if isinstance(evi_list, list) else ""
            for evi_list in x  # for each evidence list
        ][:1] if isinstance(x, list) else [])

    return df

### Part3-1.Train By Bert : 超參數 

Hyperparams

In [48]:
MODEL_NAME = "bert-base-chinese"  #@param {type:"string"}
TRAIN_BATCH_SIZE = 25  #@param {type:"integer"}
TEST_BATCH_SIZE = 25  #@param {type:"integer"}
LR = 1e-5  #@param {type:"number"}
NUM_EPOCHS = 20  #@param {type:"integer"}
MAX_SEQ_LEN = 256  #@param {type:"integer"}
EVIDENCE_TOPK = 5  #@param {type:"integer"}



In [49]:
### 檢查

EXP_DIR = f"claim_verification/e{NUM_EPOCHS}_bs{TRAIN_BATCH_SIZE}_" + f"{LR}_top{EVIDENCE_TOPK}_data{Use_Data_From_Part_1}_SEP{SEP_NUMBER}"
LOG_DIR = "logs/" + EXP_DIR
CKPT_DIR = "checkpoints/" + EXP_DIR

if not Path(LOG_DIR).exists():
    Path(LOG_DIR).mkdir(parents=True)

if not Path(CKPT_DIR).exists():
    Path(CKPT_DIR).mkdir(parents=True)

print("Save Model at : ",EXP_DIR)
print("[IMPORTANT] record Part3 Model Path to excel")

Save Model at :  claim_verification/e20_bs25_1e-05_top5_data4_SEP0
[IMPORTANT] record Part3 Model Path to excel


In [50]:
if not TRAIN_PKL_FILE.exists():
    train_df = join_with_topk_evidence(
        pd.DataFrame(TRAIN_DATA),
        mapping,
        topk=EVIDENCE_TOPK,
    )
    train_df.to_pickle(TRAIN_PKL_FILE, protocol=4)
else:
    print("You should check to rerun or not")
    with open(TRAIN_PKL_FILE, "rb") as f:
        train_df = pickle.load(f)

if not DEV_PKL_FILE.exists():
    dev_df = join_with_topk_evidence(
        pd.DataFrame(DEV_DATA),
        mapping,
        mode="eval",
        topk=EVIDENCE_TOPK,
    )
    dev_df.to_pickle(DEV_PKL_FILE, protocol=4)
else:
    print("You should check to rerun or not")
    with open(DEV_PKL_FILE, "rb") as f:
        dev_df = pickle.load(f)

### 其中train_df的"evidence_list"會合併為一句
### dev_df不會，會是[top_n句]


### 每一個Model因為都有一個新的Trainning Model，所以要對應一個新的PKL
### 所以才會被提示rerun

You should check to rerun or not
You should check to rerun or not


In [51]:
counts = train_df["label"].value_counts()
print("Now using the following train data with 0 (Negative) and 1 (Positive)")
print(counts)

Now using the following train data with 0 (Negative) and 1 (Positive)
supports           3995
NOT ENOUGH INFO    2651
refutes            2650
Name: label, dtype: int64


### Part3-2.Train By Bert : input

In [52]:
torch.cuda.empty_cache()

In [53]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_dataset = AicupTopkEvidenceBERTDataset(
    train_df,
    tokenizer=tokenizer,
    max_length=MAX_SEQ_LEN,
)
val_dataset = AicupTopkEvidenceBERTDataset(
    dev_df,
    tokenizer=tokenizer,
    max_length=MAX_SEQ_LEN,
)

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=TRAIN_BATCH_SIZE,
)
eval_dataloader = DataLoader(val_dataset, batch_size=TEST_BATCH_SIZE)

In [54]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device(
    "cpu")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(LABEL2ID),
)
model.to(device)

optimizer = AdamW(model.parameters(), lr=LR)
num_training_steps = NUM_EPOCHS * len(train_dataloader)
lr_scheduler = set_lr_scheduler(optimizer, num_training_steps)

writer = SummaryWriter(LOG_DIR)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

### Part3-3-1. Load Model (先跑Part3至此的所有Code)(因為需要Data)

In [ ]:
### 選Model的路徑

CKPT_DIR="checkpoints/claim_verification/e20_bs32_7e-05_top5" #@param {type:"string"}

### 可以配合 ls 選Model [記得改得跟上面一樣]

! ls "checkpoints/claim_verification/e20_bs32_7e-05_top5"

In [ ]:
### 選Model名稱與 Testing

ckpt_name = "val_acc=0.5466_model.150.pt"  #@param {type:"string"}
model = load_model(model, ckpt_name, CKPT_DIR)

### Part3-3-2. Training

In [ ]:
print(f"it has {len(train_dataloader)} batchs in an epoch")

it has 581 batchs in an epoch


In [ ]:
VALIDATION_STEP = 700  #@param {type:"integer"}

Training (30 mins)

In [ ]:
progress_bar = tqdm(range(num_training_steps))
current_steps = 0

strict=False

if strict:
    best_val_acc=0.5
else :
    best_val_acc=0.4

for epoch in range(NUM_EPOCHS):
    model.train()
    
    print("[Epoch {:d}/{:d}]".format(epoch, NUM_EPOCHS))

    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        writer.add_scalar("training_loss", loss.item(), current_steps)

        y_pred = torch.argmax(outputs.logits, dim=1).tolist()
        y_true = batch["labels"].tolist()

        current_steps += 1

        if current_steps % VALIDATION_STEP == 0 and current_steps > 0:
            print("Start validation")
            val_results = run_evaluation(model, eval_dataloader, device)

            # log each metric separately to TensorBoard
            for metric_name, metric_value in val_results.items():
                print(f"{metric_name}: {metric_value}")
                writer.add_scalar(f"{metric_name}", metric_value, current_steps)

            if val_results['val_acc'] > best_val_acc:
                best_val_acc=val_results['val_acc'] 
                save_checkpoint(
                    model,
                    CKPT_DIR,
                    current_steps,
                    mark=f"val_acc={val_results['val_acc']:.4f}",
                )
                print("[New Record]")

print("Finished training!")

  0%|          | 0/11620 [00:00<?, ?it/s]

[Epoch 0/20]
[Epoch 1/20]
Start validation


  0%|          | 0/146 [00:00<?, ?it/s]

val_loss: 1.7566246516900519
val_acc: 0.43287435456110157
[Epoch 2/20]
Start validation


  0%|          | 0/146 [00:00<?, ?it/s]

val_loss: 1.9663908273389894
val_acc: 0.43201376936316693
[Epoch 3/20]
Start validation


  0%|          | 0/146 [00:00<?, ?it/s]

val_loss: 2.429179063806795
val_acc: 0.48623063683304646
[Epoch 4/20]
Start validation


  0%|          | 0/146 [00:00<?, ?it/s]

val_loss: 2.692285895347595
val_acc: 0.48709122203098104
[Epoch 5/20]


### Part3-4. Make your submission (先跑1-1前的所有code)

In [56]:
Use_Data_From_Part_1 = "4"  #@param {type:"string"}

Par2_Number_of_Sent = "5"  #@param {type:"string"}

Part2_Model_Path = "checkpoints/sent_retrieval/e20_bs64_1e-05_neg0.115_top5_data4_prob0.75_yes_yes_no_yes"  #@param {type:"string"}

# CKPT_DIR = Part2_Model_Path

In [57]:
OUTPUT_FILENAME = f"submission_{MODEL_PARA}.jsonl" #@param {type:"string"}


#### 存檔名稱

In [60]:
### 選Model的路徑

CKPT_DIR="checkpoints/sent_retrieval/e20_bs64_1e-05_neg0.115_top5_data4_prob0.75_yes_yes_no_yes" #@param {type:"string"}

### 可以配合 ls 選Model

! ls "checkpoints/sent_retrieval/e20_bs64_1e-05_neg0.115_top5_data4_prob0.75_yes_yes_no_yes"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
 F1_0.6095431119224178_Precision0.5030786201363847_model.890.pt
 F1_0.6540758268388019_Precision0.5580525471319716_model.1780.pt
 F1_0.7160320102627252_Precision0.6852587244284006_model.2670.pt
 F1_0.7210003266626941_Precision0.6829522663457691_model.5340.pt
 F1_0.737284387872533_Precision0.7035900521460097_model.890.pt
 F1_0.7470057503933136_Precision0.7300742077817898_model.890.pt
 F1_0.754868552205756_Precision0.7546229442438831_model.1780.pt
 F1_0.7552447136792517_Precision0.7571901323706383_model.890.pt
 F1_0.7557266634269273_Precision0.7575511432009626_model.1780.pt
 F1_0.7573400417066631_Precision0.7638888888888888_model.2670.pt
'val_acc=0.4819_model.380.pt'
'v

In [61]:
### 選Model名稱與 Testing

ckpt_name = 'val_acc=0.5895_model.2280.pt'  #@param {type:"string"}
model = load_model(model, ckpt_name, CKPT_DIR)


In [62]:
### 決定給Part3的檔案名稱

MODEL_PARA=CKPT_DIR.split("/")
MODEL_PARA=MODEL_PARA[-1]
print("Model 參數: ",MODEL_PARA)

###
print("使用檔名(Test) : ",f"data/test_doc5sent{Par2_Number_of_Sent}_data{Use_Data_From_Part_1}_{MODEL_PARA}.jsonl")

Model 參數:  e20_bs64_1e-05_neg0.115_top5_data4_prob0.75_yes_yes_no_yes
使用檔名(Test) :  data/test_doc5sent5_data4_e20_bs64_1e-05_neg0.115_top5_data4_prob0.75_yes_yes_no_yes.jsonl


In [63]:
### 選 Part 2 出來的資料集

test_file=f"data/test_doc5sent{Par2_Number_of_Sent}_data{Use_Data_From_Part_1}_{MODEL_PARA}.jsonl"
test_file_pkl=f"data/test_doc5sent{Par2_Number_of_Sent}_data{Use_Data_From_Part_1}_{MODEL_PARA}.pkl"

TEST_DATA = load_json(test_file) 
TEST_PKL_FILE = Path(test_file_pkl)

In [64]:

if not TEST_PKL_FILE.exists():
    test_df = join_with_topk_evidence(
        pd.DataFrame(TEST_DATA),
        mapping,
        mode="eval",
        topk=EVIDENCE_TOPK,
    )
    test_df.to_pickle(TEST_PKL_FILE, protocol=4)
else:
    print("You should check to rerun or not")
    with open(TEST_PKL_FILE, "rb") as f:
        test_df = pickle.load(f)

test_dataset = AicupTopkEvidenceBERTDataset(
    test_df,
    tokenizer=tokenizer,
    max_length=MAX_SEQ_LEN,
)
test_dataloader = DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE)

Extracting evidence_list for the eval mode ...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Prediction

In [65]:
predicted_label = run_predict(model, test_dataloader, device)

Predicting:   0%|          | 0/40 [00:00<?, ?it/s]

In [66]:
### 存Testing檔

predict_dataset = test_df.copy()
predict_dataset["predicted_label"] = list(map(ID2LABEL.get, predicted_label))
predict_dataset[["id", "predicted_label", "predicted_evidence"]].to_json(
    OUTPUT_FILENAME,
    orient="records",
    lines=True,
    force_ascii=False,
)

In [67]:
print(OUTPUT_FILENAME)

submission_e20_bs64_1e-05_neg0.115_top5_data4_prob0.75_yes_yes_no_yes.jsonl


### Part3-5. XGBoost (先略過)

In [ ]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
import numpy as np

In [ ]:
### for XGBoost
from sklearn.decomposition import PCA

def Prepare_XGBoost_Data(dataframe):
    """做資料"""

    concate=[]

    for i in range(dataframe.shape[0]):
      data="[CLS] "+dataframe["claim"][i]+" [SEP] "+dataframe["evidence_list"][i]
      concate.append(data)
      
    dataframe["concate"]=concate


def Go_Through_SBERT(dataframe,dimention): 
    """做資料"""
    # Load a pre-trained Chinese SBERT model
    model = SentenceTransformer('bert-base-chinese')    
    sentences=dataframe["concate"]
    embeddings = model.encode(sentences)

    # return embeddings,np.array(dataframe["label"])

    pca = PCA(n_components=dimention)
    embeddings_reduced = pca.fit_transform(embeddings)

    return embeddings_reduced , np.array(dataframe["label"])

def Go_Through_SBERT_test(dataframe): 
    """做資料"""
    # Load a pre-trained Chinese SBERT model
    model = SentenceTransformer('bert-base-chinese')    
    sentences=dataframe["concate"]
    embeddings = model.encode(sentences)

    return embeddings

In [ ]:
if not TRAIN_PKL_FILE.exists():
    train_df = join_with_topk_evidence(
        pd.DataFrame(TRAIN_DATA),
        mapping,
        topk=EVIDENCE_TOPK,
    )
    train_df.to_pickle(TRAIN_PKL_FILE, protocol=4)
else:
    with open(TRAIN_PKL_FILE, "rb") as f:
        train_df = pickle.load(f)

if not DEV_PKL_FILE.exists():
    dev_df = join_with_topk_evidence(
        pd.DataFrame(DEV_DATA),
        mapping,
        mode="eval",
        topk=EVIDENCE_TOPK,
    )
    dev_df.to_pickle(DEV_PKL_FILE, protocol=4)
else:
    with open(DEV_PKL_FILE, "rb") as f:
        dev_df = pickle.load(f)

### 其中train_df的"evidence_list"會合併為一句
### dev_df不會，會是[top_n句]

In [ ]:
print(train_df.shape)
print(dev_df.shape)

In [ ]:
result = train_df["evidence_list"].map(lambda x: x[0])
train_df["evidence_list"]=result
train_df.head()["evidence_list"]

In [ ]:
### Make Piece of XGBoost data

Prepare_XGBoost_Data(train_df)

train_df.head()
###

In [ ]:
### Make Piece of XGBoost data

new_df=train_df.iloc[:].reset_index(drop=True)


X_train,Y_train=Go_Through_SBERT(new_df,350)
print(type(X_train),type(Y_train))
print(X_train.shape,Y_train.shape)
Sbert_df=pd.DataFrame(X_train)

print(X_train[0,:10])

###

In [ ]:
print(train_df["label"].unique())

In [ ]:
temp_y=pd.Series(Y_train)
temp_y=temp_y.map({'NOT ENOUGH INFO':0,'refutes':1,'supports':2})
# print(temp_y[:20])
temp_y.to_csv("Part_3_trainning_3_y.csv")

Y_train=temp_y.values

In [ ]:
print(Y_train)

In [ ]:
### Make Piece of XGBoost data

Sbert_df.to_csv("Part_3_trainning_3_x.csv")

###

In [ ]:
from xgboost import XGBClassifier

# 建立XGBClassifier模型
xgboostModel = XGBClassifier(n_estimators=150, learning_rate= 0.2,max_depth=5) #max_depth
# 使用訓練資料訓練模型
xgboostModel.fit(X_train, Y_train)


In [ ]:
print('訓練集: ',xgboostModel.score(X_train,Y_train))

In [ ]:
dev_df.iloc[2]["evidence_list"]

In [ ]:
result = dev_df["evidence_list"].map(lambda x: " ".join(x))
dev_df["evidence_list"]=result
# dev_df.head()["evidence_list"]
print(dev_df.iloc[:10]["evidence_list"].values)

In [ ]:
### Make Piece of XGBoost data

Prepare_XGBoost_Data(dev_df)

dev_df.head()
###

In [ ]:
### Make Piece of XGBoost data

new_df=dev_df.iloc[:].reset_index(drop=True)


X_test,Y_test=Go_Through_SBERT(new_df,350)
print(X_test.shape,Y_test.shape)
Sbert_df=pd.DataFrame(X_test)

print(X_test[0,:10])

###

In [ ]:
temp_y=pd.Series(Y_test)
temp_y=temp_y.map({'NOT ENOUGH INFO':0,'refutes':1,'supports':2})
# print(temp_y[:20])
temp_y.to_csv("Part_3_testing_3_y.csv")

Y_test=temp_y.values

In [ ]:
### Make Piece of XGBoost data

Sbert_df.to_csv("Part_3_testing_3_x.csv")

###

In [ ]:
print('訓練集: ',xgboostModel.score(X_test,Y_test))

In [ ]:
from xgboost import XGBClassifier
import pickle

best_n_tree=0
best_max_depth=0
best_model="None"
best_test_acc = 0.43

for n_tree in [25,30,40,45,50,75,100,110,125,150,180,210]:
  for depth in [3,4,5,6,7,8]:
    xgboostModel = XGBClassifier(n_estimators=n_tree, learning_rate= 0.2,max_depth=depth) #max_depth
    xgboostModel.fit(X_train, Y_train)

    test_score=xgboostModel.score(X_test,Y_test)

    print('Train: ',xgboostModel.score(X_train,Y_train),' Test: ',test_score, " With n_tree=",n_tree," ;depth=",depth,end="   ")

    if test_score > best_test_acc:
      best_model=xgboostModel
      best_n_tree=n_tree
      best_max_depth=depth
      best_test_acc=test_score
      print("[ New Record ]",end=" ")
      with open('xgboost_model_'+str(n_tree)+'_'+str(depth)+'.pkl', 'wb') as f:
        pickle.dump(xgboostModel, f)
    

    print("")

print("\n\nBest model:")
print("Best n_tree : ", best_n_tree)
print("Best max_depth : ", best_max_depth)

In [ ]:
pwd

In [ ]:
!ls | grep pkl

In [ ]:
# 載入模型

import pickle

with open('xgboost_model_40_5.pkl', 'rb') as f:
    xgboostModel = pickle.load(f)

In [ ]:
print(test_df.iloc[0]["evidence_list"].values)
test_df.head()

In [ ]:
result = test_df["evidence_list"].map(lambda x: " ".join(x))
test_df["evidence_list"]=result
# dev_df.head()["evidence_list"]
print(test_df.iloc[:10]["evidence_list"].values)

In [ ]:
### Make Piece of XGBoost data

Prepare_XGBoost_Data(test_df)

test_df.head()
###

In [ ]:
### Make Piece of XGBoost data

new_df=test_df.iloc[:].reset_index(drop=True)


X_test=Go_Through_SBERT_test(test_df)
print(X_test.shape)
Sbert_df=pd.DataFrame(X_test)

print(X_test[0,:10])

###

In [ ]:
### Make Piece of XGBoost data

Sbert_df.to_csv("Part_3_testing_3_x_data1.csv")

###

In [ ]:
proba = xgboostModel.predict_proba(X_test)
print(proba.shape)
 



In [ ]:
print(proba[:10,:])